### Attention Is All You Need (2017) 실습

본 코드는 논문 Attention Is All You Need(2017)을 리뷰한 이후 transfomer의 아키텍처를 더욱 이해하고자 실제 실습한 내용을 담고 있다.   
리뷰 내용 :    
   





* ref : https://github.com/ndb796/Deep-Learning-Paper-Review-and-Practice/blob/master/code_practices/Attention_is_All_You_Need_Tutorial_(German_English).ipynb

In [ ]:
%pip install torchtext==0.6.0 #torchtext 라이브러리 : 텍스트 처리 및 데이터 셋 로딩을 위해 사용

In [2]:
%%capture
%pip install spacy #자연어처리를 위한 라이브러리

In [ ]:
import spacy

!python -m spacy download en_core_web_sm # 영어와 독일어 모델 다운로드(토큰화)
!python -m spacy download de_core_news_sm

spacy_en = spacy.load('en_core_web_sm') 
spacy_de = spacy.load('de_core_news_sm') 

In [12]:
# 간단히 토큰화(tokenization) 기능 써보기
tokenized = spacy_en.tokenizer("I am a graduate student.")

for i, token in enumerate(tokenized):
    print(f"인덱스 {i}: {token.text}")

인덱스 0: I
인덱스 1: am
인덱스 2: a
인덱스 3: graduate
인덱스 4: student
인덱스 5: .


In [ ]:
# 영와 독일어의 토큰화 함수(순서를 뒤집지 않음) 생성
